# Experiments

### Setup

In [1]:
# You can set them inline
import os
os.environ["OPENAI_API_KEY"] = ""
os.environ["LANGSMITH_API_KEY"] = ""
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "langsmith-academy"

In [2]:
# Or you can use a .env file
from dotenv import load_dotenv
load_dotenv(dotenv_path="D:\lakshya\intro-to-langsmith\ls-academy\.env", override=True)

<>:3: SyntaxWarning: invalid escape sequence '\l'
<>:3: SyntaxWarning: invalid escape sequence '\l'
C:\Users\Lakshya\AppData\Local\Temp\ipykernel_9920\3052250427.py:3: SyntaxWarning: invalid escape sequence '\l'
  load_dotenv(dotenv_path="D:\lakshya\intro-to-langsmith\ls-academy\.env", override=True)


True

Here is the RAG Application that we've been working with throughout this course

In [3]:
import os
import tempfile
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders.sitemap import SitemapLoader
from langchain_community.vectorstores import SKLearnVectorStore
from langchain_openai import OpenAIEmbeddings
from langsmith import traceable
from openai import OpenAI
from typing import List
import nest_asyncio

# TODO: Configure this model!
MODEL_NAME = "gpt-4o"
MODEL_PROVIDER = "openai"
APP_VERSION = 1.0
RAG_SYSTEM_PROMPT = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the latest question in the conversation. 
If you don't know the answer, just say that you don't know. 
Use three sentences maximum and keep the answer concise.
"""

openai_client = OpenAI()

def get_vector_db_retriever():
    persist_path = os.path.join(tempfile.gettempdir(), "union.parquet")
    embd = OpenAIEmbeddings()

    # If vector store exists, then load it
    if os.path.exists(persist_path):
        vectorstore = SKLearnVectorStore(
            embedding=embd,
            persist_path=persist_path,
            serializer="parquet"
        )
        return vectorstore.as_retriever(lambda_mult=0)

    # Otherwise, index LangSmith documents and create new vector store
    ls_docs_sitemap_loader = SitemapLoader(web_path="https://docs.smith.langchain.com/sitemap.xml", continue_on_failure=True)
    ls_docs = ls_docs_sitemap_loader.load()

    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=500, chunk_overlap=0
    )
    doc_splits = text_splitter.split_documents(ls_docs)

    vectorstore = SKLearnVectorStore.from_documents(
        documents=doc_splits,
        embedding=embd,
        persist_path=persist_path,
        serializer="parquet"
    )
    vectorstore.persist()
    return vectorstore.as_retriever(lambda_mult=0)

nest_asyncio.apply()
retriever = get_vector_db_retriever()

"""
retrieve_documents
- Returns documents fetched from a vectorstore based on the user's question
"""
@traceable(run_type="chain")
def retrieve_documents(question: str):
    return retriever.invoke(question)

"""
generate_response
- Calls `call_openai` to generate a model response after formatting inputs
"""
@traceable(run_type="chain")
def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    messages = [
        {
            "role": "system",
            "content": RAG_SYSTEM_PROMPT
        },
        {
            "role": "user",
            "content": f"Context: {formatted_docs} \n\n Question: {question}"
        }
    ]
    return call_openai(messages)

"""
call_openai
- Returns the chat completion output from OpenAI
"""
@traceable(
    run_type="llm",
    metadata={
        "ls_provider": MODEL_PROVIDER,
        "ls_model_name": MODEL_NAME
    }
)
def call_openai(messages: List[dict]) -> str:
    return openai_client.chat.completions.create(
        model=MODEL_NAME,
        messages=messages,
    )

"""
langsmith_rag
- Calls `retrieve_documents` to fetch documents
- Calls `generate_response` to generate a response based on the fetched documents
- Returns the model response
"""
@traceable(run_type="chain")
def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    return response.choices[0].message.content


USER_AGENT environment variable not set, consider setting it to identify your requests.


### Experiment

Here is a code snippet that should look similar to what you see from the starter code!

There are a few important components here.

1. We have defined an Evaluator
2. We pipe our dataset examples (dict) to the shape of input that our function `langsmith_rag` takes (str) using a target function

In [5]:
import os
from langsmith import evaluate, Client
from langsmith.utils import LangSmithAuthError

# Set your API key programmatically (alternative to environment variables)
os.environ["LANGCHAIN_API_KEY"] = "LANGCHAIN_API_KEY"

try:
    client = Client()
    
    # Test authentication first
    try:
        client.whoami()
        print("Authentication successful!")
    except LangSmithAuthError:
        print("Authentication failed. Please check your LANGCHAIN_API_KEY environment variable.")
        exit(1)

    dataset_name = "RAG Application Golden Dataset"

    def is_concise_enough(reference_outputs: dict, outputs: dict) -> dict:
        score = len(outputs["output"]) < 1.5 * len(reference_outputs["output"])
        return {"key": "is_concise", "score": int(score)}

    def target_function(inputs: dict):
        # Make sure langsmith_rag is defined
        return langsmith_rag(inputs["question"])

    # Run evaluation
    evaluate(
        target_function,
        data=dataset_name,
        evaluators=[is_concise_enough],
        experiment_prefix="gpt-4o"
    )

except LangSmithAuthError as e:
    print(f"Authentication error: {e}")
    print("Please ensure you have set the LANGCHAIN_API_KEY environment variable correctly.")
except Exception as e:
    print(f"An error occurred: {e}")

An error occurred: 'Client' object has no attribute 'whoami'


### Modifying your Application

Now, let's change our model to gpt-35-turbo and see how it performs!

Make this change, and then run this code snippet!

In [ ]:
import os
from langsmith import evaluate
from langsmith.schemas import Example, Run

# Make sure authentication is set
os.environ["LANGCHAIN_API_KEY"] = "LANGCHAIN_API_KEY"

def target_function(inputs: dict):
    # Make sure langsmith_rag is defined
    return langsmith_rag(inputs["question"])

# Make sure these variables are defined
dataset_name = "RAG Application Golden Dataset"  # or your actual dataset name
is_concise_enough = ...  # your evaluator function

try:
    evaluate(
        target_function,
        data=dataset_name,
        evaluators=[is_concise_enough],
        experiment_prefix="gpt-3.5-turbo"
    )
    print("Evaluation completed successfully!")
except Exception as e:
    print(f"Evaluation failed: {e}")

Evaluation failed: Authentication failed for /datasets. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/datasets?limit=1&name=RAG+Application+Golden+Dataset', '{"detail":"Invalid token"}')


### Running over Different pieces of Data

##### Dataset Version

You can execute an experiment on a specific version of a dataset in the sdk by using the `as_of` parameter in `list_examples`

Let's try running on just our initial dataset.

In [ ]:
from langsmith import evaluate, Client
from langsmith.schemas import Example, Run

def target_function(inputs: dict):
    return langsmith_rag(inputs["question"])

evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="gpt-3.5-turbo"
)

LangSmithAuthError: Authentication failed for /datasets. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/datasets?limit=1&name=RAG+Application+Golden+Dataset', '{"detail":"Invalid token"}')

##### Dataset Split

You can run an experiment on a specific split of your dataset, let's try running on the Crucial Examples split.

In [ ]:
evaluate(
    target_function,
    data=client.list_examples(
        dataset_name=dataset_name, 
        example_ids=[   # We pass in a specific list of example_ids
            
            "29a11b81-320f-4df7-a653-ecd18de5e3b7",
            "8acfbb7a-89ce-4a94-aec9-07927e37350c"
        ]
    ),
    evaluators=[is_concise_enough],
    experiment_prefix="two specific example ids"
)

##### Specific Data Points

You can specify individual data points to run an experiment over as well

In [ ]:
evaluate(
    target_function,
    data=client.list_examples(
        dataset_name=dataset_name, 
        example_ids=[   # We pass in a specific list of example_ids
            
            "29a11b81-320f-4df7-a653-ecd18de5e3b7",
            "8acfbb7a-89ce-4a94-aec9-07927e37350c"
        ]
    ),
    evaluators=[is_concise_enough],
    experiment_prefix="two specific example ids"
)

### Other Parameters

##### Repetitions

You can run an experiment several times to make sure you have consistent results

In [ ]:
evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="two repetitions",
    num_repetitions=2   # This field defaults to 1
)

##### Concurrency
You can also kick off concurrent threads of execution to make your experiments finish faster!

In [ ]:
evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="concurrency",
    max_concurrency=3,  # This defaults to None, so this is an improvement!
)

##### Metadata 

You can (and should) add metadata to your experiments, to make them easier to find in the UI

In [ ]:
evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="metadata added",
    metadata={  # We can pass custom metadata for the experiment, such as the model name
        "model_name": MODEL_NAME
    }
)